In [5]:
dataset_id = "JHU_Daily_Reports_refined"

In [6]:
dataset_id2 = "US_Census_Bureau_refined"

In [7]:
!bq --location=US mk --dataset {dataset_id}

Dataset 'lucid-hook-288402:JHU_Daily_Reports_refined' successfully created.


In [8]:
!bq --location=US mk --dataset {dataset_id2}

Dataset 'lucid-hook-288402:US_Census_Bureau_refined' successfully created.


In [9]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id}.COVID19_USdeaths 'gs://utbeemers_covid19/covid19/time_series_covid_19_deaths_US.csv'

Waiting on bqjob_rb8c1e8fcc4b2a02_00000175d359edf7_1 ... (4s) Current status: DONE   


In [10]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id}.COVID19_Globaldeaths 'gs://utbeemers_covid19/covid19/time_series_covid_19_deaths_global.csv'

Waiting on bqjob_r3c452f5a78c11811_00000175d35b3f14_1 ... (2s) Current status: DONE   


In [11]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id2}.Peopleover65_USdeaths 'gs://utbeemers_covid19/secondary dataset/GCT0103.csv'

Waiting on bqjob_r7ae9224966fca21_00000175d35b7ae0_1 ... (1s) Current status: DONE   


In [12]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id2}.Peopleover85_USdeaths 'gs://utbeemers_covid19/secondary dataset/GCT0104.csv'

Waiting on bqjob_r505ca5821404f95c_00000175d35bc27b_1 ... (1s) Current status: DONE   


In [8]:
# Split Table COVID19_USDeaths into 2 different entities for geospots and US_populations

In [7]:
%%bigquery
create table JHU_Daily_Reports_refined.COVID19_USgeohotspots 
as select UID, Lat as latitude, Long_ as longitude
from JHU_Daily_Reports_staging.COVID19_USdeaths
group by UID, Lat, Long_ 
having Lat != 0 and Long_ != 0

Empty DataFrame
Columns: []
Index: []

In [10]:
%%bigquery
create table JHU_Daily_Reports_refined.COVID19_USPopulations 
as select UID, Population
from JHU_Daily_Reports_staging.COVID19_USdeaths
group by UID, Population 
having Population != 0 

Empty DataFrame
Columns: []
Index: []

In [41]:
%%bigquery
create table JHU_Daily_Reports_refined.COVID19_USdeaths as select * except(Lat, Long_, Population) from JHU_Daily_Reports_staging.COVID19_USdeaths

Empty DataFrame
Columns: []
Index: []

In [27]:
%%bigquery
select * from JHU_Daily_Reports_refined.COVID19_USgeohotspots


UID   latitude   longitude
0     84020165  38.522753  -99.308640
1     84005103  33.588398  -92.877960
2     84019043  42.844434  -91.341337
3     84016043  44.228009 -111.480994
4     84040113  36.628888  -96.396357
...        ...        ...         ...
3227  84049015  38.996171 -110.701396
3228  84051073  37.408090  -76.535243
3229  84006051  37.938993 -118.887241
3230  84026057  43.292656  -84.605054
3231  84054065  39.554390  -78.263117

[3232 rows x 3 columns]

In [11]:
%%bigquery
select * from JHU_Daily_Reports_refined.COVID19_USPopulations

UID  Population
0     84048235        1536
1     84054017        8448
2     84017185       11520
3     84036015       83456
4     84016005       87808
...        ...         ...
3220  84028061       16383
3221  84028139       22015
3222  84029001       25343
3223  84022047       32511
3224  84024013      168447

[3225 rows x 2 columns]

In [37]:
%%bigquery
select * except(Lat, Long_) from JHU_Daily_Reports_refined.COVID19_USdeaths

UID iso2 iso3  code3     FIPS      Admin2  Province_State  \
0     84001045   US  USA    840   1045.0        Dale         Alabama   
1     84090017   US  USA    840  90017.0  Unassigned        Illinois   
2     84037087   US  USA    840  37087.0     Haywood  North Carolina   
3     84038015   US  USA    840  38015.0    Burleigh    North Dakota   
4     84047179   US  USA    840  47179.0  Washington       Tennessee   
...        ...  ...  ...    ...      ...         ...             ...   
3335  84017029   US  USA    840  17029.0       Coles        Illinois   
3336  84037139   US  USA    840  37139.0  Pasquotank  North Carolina   
3337  84013321   US  USA    840  13321.0       Worth         Georgia   
3338  84039123   US  USA    840  39123.0      Ottawa            Ohio   
3339  84026093   US  USA    840  26093.0  Livingston        Michigan   

     Country_Region                    Combined_Key  Population  ...  \
0                US               Dale, Alabama, US       49172  ...   
1                US        Unassigned, Illinois, US           0  ...   
2                US     Haywood, North Carolina, US       62317  ...   
3                US      Burleigh, North Dakota, US       95626  ...   
4                US       Washington, Tennessee, US      129375  ...   
...             ...                             ...         ...  ...   
3335             US             Coles, Illinois, US       50621  ...   
3336             US  Pasquotank, North Carolina, US       39824  ...   
3337             US              Worth, Georgia, US       20247  ...   
3338             US                Ottawa, Ohio, US       40525  ...   
3339             US        Livingston, Michigan, US      191995  ...   

      _10_25_20  _10_26_20  _10_27_20  _10_28_20  _10_29_20  _10_30_20  \
0            51         51         52         52         52         52   
1           270        270        270        270        270        283   
2            36         36         36         36         36         36   
3            76         76         77         78         78         78   
4            51         51         53         53         53         58   
...         ...        ...        ...        ...        ...        ...   
3335         39         39         40         40         40         40   
3336         35         35         35         35         35         35   
3337         32         32         32         32         33         34   
3338         30         30         30         30         30         30   
3339         33         33         33         33         35         35   

      _10_31_20  _11_1_20  _11_2_20  _11_3_20  
0            52        52        52        52  
1           283       283       283       283  
2            36        36        36        36  
3            78        79        80        83  
4            59        59        60        67  
...         ...       ...       ...       ...  
3335         40        40        40        42  
3336         35        35        35        35  
3337         35        35        35        35  
3338         30        30        30        30  
3339         35        35        35        35  

[3340 rows x 297 columns]

In [10]:
# Use Cast and String functions to adjust date values from type INTEGER to DATE

In [11]:
# Note: There are hundreds of columns that require casting from INTEGER to DATE. Under the TA's
# recommendation, this will simply be noted for this milestone, and in future milestones, the data 
# will be adjusted so there are no longer hundreds of columns.

In [12]:
# Ensure that there are no duplicate primary keys in the first dataset.
# The following two queries will return the same count if there are no duplicate
# entries in the primary key field (UID).

In [29]:
%%bigquery
SELECT COUNT(UID) UID_distinct_count FROM
  (SELECT DISTINCT UID
  FROM JHU_Daily_Reports_refined.COVID19_USdeaths);

UID_distinct_count
0                3340

In [30]:
%%bigquery
SELECT COUNT(UID) UID_count FROM
  (SELECT UID
  FROM JHU_Daily_Reports_refined.COVID19_USdeaths);

UID_count
0       3340

In [45]:
# Ensure that there are no duplicate primary keys in the secondary dataset.
# The following two queries will return the same count if there are no duplicate
# entries in the primary key field (GEOID).

In [32]:
%%bigquery
SELECT COUNT(GEOID) GEOID_distinct_count FROM
  (SELECT DISTINCT GEOID
  FROM US_Census_Bureau_refined.Peopleover65_USdeaths);

GEOID_distinct_count
0                  7283

In [33]:
%%bigquery
SELECT COUNT(GEOID) GEOID_count FROM
  (SELECT GEOID
  FROM US_Census_Bureau_refined.Peopleover65_USdeaths);

GEOID_count
0         7283

In [43]:
%%bigquery
SELECT COUNT(GEOID) GEOID_distinct_count FROM
  (SELECT DISTINCT GEOID
  FROM US_Census_Bureau_refined.Peopleover85_USdeaths);

GEOID_distinct_count
0                  7283

In [44]:
%%bigquery
SELECT COUNT(GEOID) GEOID_count FROM
  (SELECT GEOID
  FROM US_Census_Bureau_refined.Peopleover85_USdeaths);

GEOID_count
0         7283

In [20]:
# 5 SQL Queries

In [22]:
#1- aggregation - showing the average number of predicted deaths across all of the US states in order for _7_14_20

In [18]:
%%bigquery
select Province_State, avg(_7_14_20) as deaths from JHU_Daily_Reports_refined.COVID19_USdeaths group by Province_State order by Province_State

Province_State      deaths
0                    Alabama   16.869565
1                     Alaska    0.548387
2             American Samoa    0.000000
3                    Arizona  137.470588
4                   Arkansas    4.298701
5                 California  120.833333
6                   Colorado   26.333333
7                Connecticut  437.200000
8                   Delaware  103.600000
9           Diamond Princess    0.000000
10      District of Columbia  189.333333
11                   Florida   63.898551
12                   Georgia   18.968944
13            Grand Princess    3.000000
14                      Guam    5.000000
15                    Hawaii    3.142857
16                     Idaho    2.239130
17                  Illinois   71.336538
18                   Indiana   29.521277
19                      Iowa    7.495050
20                    Kansas    2.794393
21                  Kentucky    5.204918
22                 Louisiana   52.196970
23                     Maine    6.333333
24                  Maryland  128.230769
25             Massachusetts  490.588235
26                  Michigan   72.712644
27                 Minnesota   17.393258
28               Mississippi   15.142857
29                  Missouri    9.440678
30                   Montana    0.586207
31                  Nebraska    3.010526
32                    Nevada   32.210526
33             New Hampshire   32.666667
34                New Jersey  677.478261
35                New Mexico   15.742857
36                  New York  506.703125
37            North Carolina   15.401961
38              North Dakota    1.600000
39  Northern Mariana Islands    2.000000
40                      Ohio   34.100000
41                  Oklahoma    5.417722
42                    Oregon    6.421053
43              Pennsylvania  100.449275
44               Puerto Rico    2.112500
45              Rhode Island  141.000000
46            South Carolina   20.687500
47              South Dakota    1.602941
48                 Tennessee    7.907216
49                     Texas   15.394531
50                      Utah    6.108108
51                   Vermont    3.500000
52            Virgin Islands    6.000000
53                  Virginia   14.644444
54                Washington   34.243902
55             West Virginia    1.701754
56                 Wisconsin   11.162162
57                   Wyoming    0.880000

In [26]:
#2- showing the regions in US where the percent of population over 85 is greater than 1.5% and ordering starting from max percentage 

In [25]:
%%bigquery
select ESTIMATE as percent_of_pop_over_85, GEONAME from US_Census_Bureau_refined.Peopleover85_USdeaths where ESTIMATE > 1.5 order by ESTIMATE DESC limit 20

percent_of_pop_over_85                                            GEONAME
0                      7.8  Palm Beach County (Southeast)--Greater Delray ...
1                      7.6  Sarasota County (Central & Gulf Coast) PUMA, F...
2                      7.5  Maricopa County--El Mirage City & Sun City PUM...
3                      6.9  Lee County (South & West)--Barrier Islands Reg...
4                      6.7       Sebring--Avon Park, FL Urbanized Area (2010)
5                      6.6  Palm Beach County (West of Boca Raton City) PU...
6                      6.4           Collier County (Northwest) PUMA, Florida
7                      6.1                              Tamarac city, Florida
8                      5.8  Honolulu County--East Honolulu to Kapahulu PUM...
9                      5.7                          Charlotte County, Florida
10                     5.7                         Punta Gorda, FL Metro Area
11                     5.7          Charlotte County School District, Florida
12                     5.7                     Charlotte County PUMA, Florida
13                     5.5           Collier County (Southwest) PUMA, Florida
14                     5.5  Palm Beach County (Southeast)--Boca Raton City...
15                     5.5  Lady Lake--The Villages, FL Urbanized Area (2010)
16                     5.5  Miami-Dade County (Northeast)--Hialeah City (S...
17                     5.5  Broward County--Pompano Beach (South) & Fort L...
18                     5.4           Bonita Springs, FL Urbanized Area (2010)
19                     5.3    Williamsville Central School District, New York

In [6]:
# Subquery- Listing thirty regions in Texas and their corresponding population among the people they have through different entities

In [42]:
%%bigquery
select distinct Admin2, (select distinct Population from JHU_Daily_Reports_refined.COVID19_USPopulations b where b.UID= a.UID) as population 
from JHU_Daily_Reports_refined.COVID19_USdeaths a 
where Province_State = 'Texas'
limit 30

Admin2  population
0      Out of TX         NaN
1     Unassigned         NaN
2         Archer      8553.0
3            Bee     32565.0
4         Borden       654.0
5         Carson      5926.0
6       Colorado     21493.0
7          Cooke     41257.0
8       Crockett      3464.0
9         Donley      3278.0
10       Edwards      1932.0
11         Falls     17297.0
12     Freestone     19717.0
13         Garza      6229.0
14       Houston     22968.0
15    Hutchinson     20938.0
16     Jim Wells     40482.0
17      Lampasas     21428.0
18          Leon     17404.0
19      Live Oak     12207.0
20       Madison     14284.0
21        Motley      1200.0
22         Nolan     14714.0
23      Presidio      6704.0
24          Real      3452.0
25       Roberts       854.0
26  San Patricio     66730.0
27   Shackelford      3265.0
28     Somervell      9128.0
29     Stonewall      1350.0

In [1]:
# Aggregated (correlated) Subquery- Listing which Regions+States have a higher percent of poeple over 85 than average percent of poeple over 85 in that Regions+States of their error margin 

In [50]:
%%bigquery
select a.ESTIMATE, a.GEONAME
from US_Census_Bureau_refined.Peopleover85_USdeaths a
where a.ESTIMATE > (select avg(ESTIMATE) from US_Census_Bureau_refined.Peopleover85_USdeaths b where a.MG_ERROR= B.MG_ERROR) 
order by a.ESTIMATE DESC limit 100

ESTIMATE                                            GEONAME
0        7.8  Palm Beach County (Southeast)--Greater Delray ...
1        7.6  Sarasota County (Central & Gulf Coast) PUMA, F...
2        7.5  Maricopa County--El Mirage City & Sun City PUM...
3        6.9  Lee County (South & West)--Barrier Islands Reg...
4        6.7       Sebring--Avon Park, FL Urbanized Area (2010)
..       ...                                                ...
95       4.0                    Carolina Municipio, Puerto Rico
96       4.0             Wheeling, WV--OH Urbanized Area (2010)
97       4.0         Northfield township, Cook County, Illinois
98       4.0         Bayamón Municipio (Este) PUMA, Puerto Rico
99       4.0      Middlesex County (Southeast) PUMA, New Jersey

[100 rows x 2 columns]

In [7]:
# Subquery- Listing the regions in Kentucky and their corresponding x_coordinate of latitude among their location through two different entities

In [8]:
%%bigquery
select distinct a.Admin2, (select latitude from JHU_Daily_Reports_refined.COVID19_USgeohotspots b where b.UID= a.UID) as x_coordinate 
from JHU_Daily_Reports_refined.COVID19_USdeaths a 
where Province_State = 'Kentucky'
order by a.Admin2

Admin2  x_coordinate
0       Adair     37.104598
1       Allen     36.751976
2    Anderson     38.006710
3     Ballard     37.058428
4      Barren     36.964698
..        ...           ...
117     Wayne     36.803107
118   Webster     37.521684
119   Whitley     36.757005
120     Wolfe     37.739416
121  Woodford     38.041782

[122 rows x 2 columns]

In [13]:
# Create Views for data studio queries

In [30]:
%%bigquery
CREATE VIEW JHU_Daily_Reports_refined.v_Average_Deaths_On_7_14_20_per_state AS
select Province_State, avg(_7_14_20) as deaths from JHU_Daily_Reports_refined.COVID19_USdeaths group by Province_State order by deaths DESC

Empty DataFrame
Columns: []
Index: []

In [31]:
%%bigquery
CREATE VIEW US_Census_Bureau_refined.v_states_over_average_percentpeopleover85 AS
select a.ESTIMATE, a.GEONAME
from US_Census_Bureau_refined.Peopleover85_USdeaths a
where a.ESTIMATE > (select avg(ESTIMATE) from US_Census_Bureau_refined.Peopleover85_USdeaths b where a.MG_ERROR= B.MG_ERROR) 
order by a.ESTIMATE DESC limit 100

Empty DataFrame
Columns: []
Index: []